Script to build the action catalog v1 (old version, deprecated)

In [1]:
"""Build the iso-only action catalog once and save to disk.

Run this script OFFLINE (or once per job) after your graph libraries are
loaded. It deduplicates graphs by directed isomorphism, creates the lookup
(num_inputs, tt_int) → action_ids, and writes a pickle:

    action_catalog.pkl  # {"graphs": UNIQUE_GRAPHS, "lookup": TTABLE_TO_ACTIONS}

Set env var ACTION_CATALOG_PATH if you save it elsewhere.
"""
from __future__ import annotations

import pickle, warnings
from pathlib import Path
from collections import defaultdict
from typing import Dict, List, Tuple

import networkx as nx
from tqdm import tqdm
from networkx.algorithms import weisfeiler_lehman_graph_hash as _wl_hash

from utils5 import calculate_truth_table  # <- import your existing TT helper

# -----------------------------------------------------------------------------
# 1) Load graph libraries ------------------------------------------------------
# Adjust these paths to your pickles
# -----------------------------------------------------------------------------
with open("graphs_library_1_input_pruned.pkl", "rb") as f:
    graphs_lib1 = pickle.load(f)
with open("graphs_library_2_input_pruned.pkl", "rb") as f:
    graphs_lib2 = pickle.load(f)
with open("graphs_library_3_input_3_7_pruned.pkl", "rb") as f:
    graphs_lib3 = pickle.load(f)
with open("graphs_library_4_input_4_3_pruned.pkl", "rb") as f:
    graphs_lib4 = pickle.load(f)

LIBRARIES_RAW: Dict[int, object] = {
    1: graphs_lib1,
    2: graphs_lib2,
    3: graphs_lib3,
    4: graphs_lib4,
}

# Convert dict->list if needed
LIBRARIES: Dict[int, List[nx.DiGraph]] = {}
for n_in, lib in LIBRARIES_RAW.items():
    if isinstance(lib, dict):
        LIBRARIES[n_in] = list(lib.values())
    elif isinstance(lib, list):
        LIBRARIES[n_in] = lib
    else:
        raise TypeError(f"Library for {n_in}-input graphs must be list or dict")

# -----------------------------------------------------------------------------
# 2) Helpers -------------------------------------------------------------------
# -----------------------------------------------------------------------------

def _quick_hash(g: nx.DiGraph) -> str:
    return _wl_hash(g.to_undirected(), node_attr=None, edge_attr=None)

def _truth_key(g: nx.DiGraph) -> Tuple[int, int]:
    tt = calculate_truth_table(g)
    bits = "".join(str(o[0]) for _, o in sorted(tt.items()))
    num_inputs = (len(tt).bit_length() - 1)
    return (num_inputs, int(bits, 2))

# -----------------------------------------------------------------------------
# 3) Build catalog -------------------------------------------------------------
# -----------------------------------------------------------------------------
UNIQUE_GRAPHS: List[nx.DiGraph] = []
TTABLE_TO_ACTIONS: Dict[Tuple[int, int], List[int]] = defaultdict(list)
_bucket: Dict[str, List[int]] = defaultdict(list)

all_graphs = [g for lib in LIBRARIES.values() for g in lib]
non_graphs = 0
for g in tqdm(all_graphs, desc="Building catalog", unit="graph"):
    if not isinstance(g, nx.DiGraph):
        non_graphs += 1
        continue
    qh = _quick_hash(g)
    act_id = None
    for cand in _bucket[qh]:
        if nx.is_isomorphic(g, UNIQUE_GRAPHS[cand]):
            act_id = cand
            break
    if act_id is None:
        act_id = len(UNIQUE_GRAPHS)
        UNIQUE_GRAPHS.append(g)
        _bucket[qh].append(act_id)
    TTABLE_TO_ACTIONS[_truth_key(g)].append(act_id)

for k in TTABLE_TO_ACTIONS:
    TTABLE_TO_ACTIONS[k] = list(dict.fromkeys(TTABLE_TO_ACTIONS[k]))

print(f"Catalog size: {len(UNIQUE_GRAPHS)} unique iso graphs")
if non_graphs:
    warnings.warn(f"Ignored {non_graphs} non-graph entries.")

# -----------------------------------------------------------------------------
# 4) Save ----------------------------------------------------------------------
# -----------------------------------------------------------------------------
out_path = Path("action_catalog_old.pkl")
with out_path.open("wb") as f:
    pickle.dump({"graphs": UNIQUE_GRAPHS, "lookup": TTABLE_TO_ACTIONS}, f)
print(f"Catalog saved to {out_path.resolve()}")

Building catalog: 100%|██████████| 154103/154103 [07:26<00:00, 345.39graph/s]


Catalog size: 15928 unique iso graphs
Catalog saved to /home/gridsan/spalacios/DRL1/supercloud-testing/ABC-and-PPO-testing1/action_catalog.pkl


Some testing scripts

In [ ]:
import networkx as nx
from tqdm import tqdm
from collections import defaultdict

# ---- If you run this independently, import or copy from build_action_catalog.py ----
import pickle

with open("graphs_library_1_input_pruned.pkl", "rb") as f:
    graphs_lib1 = pickle.load(f)
with open("graphs_library_2_input_pruned.pkl", "rb") as f:
    graphs_lib2 = pickle.load(f)
with open("graphs_library_3_input_3_7_pruned.pkl", "rb") as f:
    graphs_lib3 = pickle.load(f)
with open("graphs_library_4_input_4_3_pruned.pkl", "rb") as f:
    graphs_lib4 = pickle.load(f)

LIBRARIES_RAW = {
    1: graphs_lib1,
    2: graphs_lib2,
    3: graphs_lib3,
    4: graphs_lib4,
}

LIBRARIES = {
    n_in: list(lib.values()) if isinstance(lib, dict) else lib
    for n_in, lib in LIBRARIES_RAW.items()
}

# ---- Flatten the graphs ----
all_graphs = [g for lib in LIBRARIES.values() for g in lib if isinstance(g, nx.DiGraph)]

# ------------------------------------------------------------------------------
# Directed Isomorphism Deduplication
# ------------------------------------------------------------------------------
print("\n[Directed Isomorphism Deduplication]")
directed_unique = []
for g in tqdm(all_graphs, desc="Checking directed isomorphisms"):
    for h in directed_unique:
        if nx.is_isomorphic(g, h):
            break
    else:
        directed_unique.append(g)

print(f"→ Found {len(directed_unique)} unique directed graphs")

# ------------------------------------------------------------------------------
# Undirected Isomorphism Deduplication
# ------------------------------------------------------------------------------
print("\n[Undirected Isomorphism Deduplication]")
undirected_unique = []
for g in tqdm(all_graphs, desc="Checking undirected isomorphisms"):
    g_undir = g.to_undirected()
    for h in undirected_unique:
        if nx.is_isomorphic(g_undir, h.to_undirected()):
            break
    else:
        undirected_unique.append(g)

print(f"→ Found {len(undirected_unique)} unique undirected graphs")

# ------------------------------------------------------------------------------
# Summary
# ------------------------------------------------------------------------------
print("\n[Summary]")
print(f"Total input graphs:       {len(all_graphs)}")
print(f"Unique (directed):        {len(directed_unique)}")
print(f"Unique (undirected):      {len(undirected_unique)}")
print(f"Graphs with identical structure when ignoring direction: {len(undirected_unique) - len(directed_unique)} more")




[Directed Isomorphism Deduplication]


Checking directed isomorphisms: 100%|██████████| 154103/154103 [3:30:27<00:00, 12.20it/s]  


→ Found 15928 unique directed graphs

[Undirected Isomorphism Deduplication]


Checking undirected isomorphisms:  14%|█▎        | 20897/154103 [20:21<4:15:33,  8.69it/s]

Script to build the action catalog v2

This version avoids missing some key --> aid mappings by not assuming that the initial libraries have all the permutations of each unique graph

In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Build a permutation-complete action catalogue.

Resulting pickle:
    action_catalog.pkl   # {"graphs": UNIQUE_GRAPHS,
                         #  "lookup": TTABLE_TO_ACTIONS}
"""

from __future__ import annotations

import pickle, warnings, itertools
from pathlib import Path
from collections import defaultdict
from typing import Dict, List, Tuple

import networkx as nx
from tqdm import tqdm
from networkx.algorithms import weisfeiler_lehman_graph_hash as _wl_hash

from utils5 import calculate_truth_table_v2     # your existing helper
# ---------------------------------------------------------------------
# 1) Load graph libraries ---------------------------------------------
# ---------------------------------------------------------------------
with open("graphs_library_1_input_pruned.pkl", "rb") as f:
    graphs_lib1 = pickle.load(f)
with open("graphs_library_2_input_pruned.pkl", "rb") as f:
    graphs_lib2 = pickle.load(f)
with open("graphs_library_3_input_3_7_pruned.pkl", "rb") as f:
    graphs_lib3 = pickle.load(f)
with open("graphs_library_4_input_4_3_pruned.pkl", "rb") as f:
    graphs_lib4 = pickle.load(f)

LIBRARIES_RAW: Dict[int, object] = {
    1: graphs_lib1,
    2: graphs_lib2,
    3: graphs_lib3,
    4: graphs_lib4,
}

# convert dict → list if needed
LIBRARIES: Dict[int, List[nx.DiGraph]] = {}
for n_in, lib in LIBRARIES_RAW.items():
    if isinstance(lib, dict):
        LIBRARIES[n_in] = list(lib.values())
    elif isinstance(lib, list):
        LIBRARIES[n_in] = lib
    else:
        raise TypeError(f"Library for {n_in}-input graphs must be list or dict")

# ---------------------------------------------------------------------
# 2) Helpers -----------------------------------------------------------
# ---------------------------------------------------------------------
def _quick_hash(g: nx.DiGraph) -> str:
    """WL hash of the undirected shadow – fast iso pre-filter."""
    return _wl_hash(g.to_undirected(), node_attr=None, edge_attr=None)

def _truth_key(g: nx.DiGraph) -> Tuple[int, int]:
    """(num_inputs, integer-encoded truth table)."""
    tt = calculate_truth_table_v2(g)           # dict[input_tuple] → [out_bit]
    bits = "".join(str(o[0]) for _, o in sorted(tt.items()))
    num_inputs = (len(tt).bit_length() - 1)
    return num_inputs, int(bits, 2)

# ---------------------------------------------------------------------
# 3) Deduplicate by isomorphism ---------------------------------------
# ---------------------------------------------------------------------
UNIQUE_GRAPHS: List[nx.DiGraph] = []
_iso_bucket: Dict[str, List[int]] = defaultdict(list)   # WL-hash → act_ids

all_graphs = [g for lib in LIBRARIES.values() for g in lib]
non_graphs = 0

for g in tqdm(all_graphs, desc="Deduplicating (iso)", unit="graph"):
    if not isinstance(g, nx.DiGraph):
        non_graphs += 1
        continue

    qh = _quick_hash(g)
    act_id = None
    for cand in _iso_bucket[qh]:
        if nx.is_isomorphic(g, UNIQUE_GRAPHS[cand]):
            act_id = cand                      # already seen iso
            break
    if act_id is None:                         # new iso-class
        act_id = len(UNIQUE_GRAPHS)
        UNIQUE_GRAPHS.append(g)
        _iso_bucket[qh].append(act_id)

print(f"Unique iso graphs: {len(UNIQUE_GRAPHS)}")
if non_graphs:
    warnings.warn(f"Ignored {non_graphs} non-graph entries.")

# ---------------------------------------------------------------------
# 4) Build permutation-complete lookup --------------------------------
# ---------------------------------------------------------------------
TTABLE_TO_ACTIONS: Dict[Tuple[int, int], List[int]] = defaultdict(list)

for aid, g in enumerate(tqdm(UNIQUE_GRAPHS,
                             desc="Storing all permutations", unit="graph")):
    pins = [n for n in g if g.in_degree(n) == 0]

    # iterate over ALL input-pin permutations
    for perm in itertools.permutations(pins):
        g_perm = nx.relabel_nodes(g, dict(zip(pins, perm)), copy=True)
        key = _truth_key(g_perm)
        TTABLE_TO_ACTIONS[key].append(aid)

# deduplicate the action lists (they’re tiny – set→list is fine)
for k, lst in TTABLE_TO_ACTIONS.items():
    TTABLE_TO_ACTIONS[k] = sorted(set(lst))

print(f"Lookup entries: {len(TTABLE_TO_ACTIONS)} (key → action list)")

# ---------------------------------------------------------------------
# 5) Save --------------------------------------------------------------
# ---------------------------------------------------------------------
out_path = Path("action_catalog.pkl")
with out_path.open("wb") as f:
    pickle.dump({"graphs": UNIQUE_GRAPHS, "lookup": TTABLE_TO_ACTIONS}, f)

print(f"Permutation-complete catalogue saved to {out_path.resolve()}")


Deduplicating (iso): 100%|██████████| 154103/154103 [06:05<00:00, 421.64graph/s]


Unique iso graphs: 15928


Storing all permutations: 100%|██████████| 15928/15928 [08:40<00:00, 30.58graph/s]


Lookup entries: 65539 (key → action list)
Permutation-complete catalogue saved to /home/gridsan/spalacios/DRL1/supercloud-testing/ABC-and-PPO-testing1/action_catalog.pkl
